# Convolutional Neural Networks

#### Author : ishandeveloper

In the previous example, we saw how we could create a neural network called a deep neural network to pattern match a set of images of fashion items to labels. 

Now, one of the things that we have have seen when we looked at the images is that there's a lot of wasted space in each image. While there are only 784 pixels, it will be interesting to see if there was a way that we could condense the image down to the important features that distinguish what makes it a shoe, or a handbag, or a shirt. That's where convolutions come in. 

Convolutions in a very basic way can be inferred to as applying filters on an image to enhance some of it's distinguishing features. For example a filter that helps to emphasize all the vertical lines in an image and another one for horizontal lines etc.

#### When these convolutions are combined with something called pooling, they can become really powerful. 

## Pooling
Simply, pooling is a way of compressing an image. A quick and easy way to do this, is to go over the image of four pixels at a time, i.e, the current pixel and its neighbors underneath and to the right of it. Of these four, pick the biggest value and keep just that. This will preserve the features that were highlighted by the convolution, while simultaneously quartering the size of the image. We have the horizontal and vertical axes.

First of all, Let's <br>
<b>Get The Dependencies</b>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy

.



Here's our code from the earlier example, where we defined out a neural network to have an input layer in the shape of our data, and output layer in the shape of the number of categories we're trying to define, and a hidden layer in the middle. The Flatten takes our square 28 by 28 images and turns them into a one dimensional array.

In [22]:
# Importing Dataset
mnist = tf.keras.datasets.fashion_mnist

# Splitting Dataset
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images / 255.0
test_images=test_images / 255.0

In [23]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation=tf.nn.relu ),
    keras.layers.Dense(10,  activation=tf.nn.softmax)
]) 

Let's try to evaluate the above neural network once again.

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

Epoch 1/5
60000/60000 [==============================] - 5s 77us/sample - loss: 0.5019 - accuracy: 0.8235
Epoch 2/5
60000/60000 [==============================] - 4s 64us/sample - loss: 0.3764 - accuracy: 0.8645
Epoch 3/5
60000/60000 [==============================] - 5s 78us/sample - loss: 0.3379 - accuracy: 0.8774s - loss:
Epoch 4/5
60000/60000 [==============================] - 5s 85us/sample - loss: 0.3115 - accuracy: 0.8876
Epoch 5/5
10000/10000 [==============================] - 0s 39us/sample - loss: 0.3570 - accuracy: 0.8727


[0.3570241296052933, 0.8727]

Take note that the above Neural Network gives us an accuracy of <b>88%</b>. Later on, we'll check how much difference does adding a CNN makes to the above accuracy.

To add CNN to this, we'll simply add all other different layers. But, first let's reshape our 

In [20]:
model = keras.Sequential([
    keras.layers.Conv2D(64, (3,3), activation='relu',input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation=tf.nn.relu ),
    keras.layers.Dense(10,  activation=tf.nn.softmax)
]) 

You'll see that the last three lines are the same, the Flatten, the Dense hidden layer with 128 neurons, and the Dense output layer with 10 neurons. What's different is what has been added on top of this. Let's take a look at this, line by line.

Here we're specifying the first convolution. We're asking keras to generate 64 filters for us. These filters are 3 by 3, their activation is relu, which means the negative values will be thrown way, and finally the input shape is as before, the 28 by 28. That extra 1 just means that we are tallying using a single byte for color depth. As we saw before our image is our gray scale, so we just use one byte.

The next line of code will then create a pooling layer. It's max-pooling because we're going to take the maximum value. We're saying it's a two-by-two pool, so for every four pixels, the biggest one will survive as shown earlier. We then add another convolutional layer, and another max-pooling layer so that the network can learn another set of convolutions on top of the existing one, and then again, pool to reduce the size. So, by the time the image gets to the flatten to go into the dense layers, it's already much smaller. It's being quartered, and then quartered again. So, its content has been greatly simplified, the goal being that the convolutions will filter it to the features that determine the output.

In [6]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               204928    
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

A really useful method on the model is the model.summary method. This allows us to inspect the layers of the model, and see the journey of the image through the convolutions, and here is the output. It's a nice table showing us the layers, and some details about them including the output shape.

It's important to keep an eye on the output shape column. When you first look at this, it can be a little bit confusing and feel like a bug. After all, isn't the data 28 by 28, so y is the output, 26 by 26. The key to this is remembering that the filter is a three by three filter.

You can't calculate the filter for the pixel in the top left, because it doesn't have any neighbors above it or to its left. In a similar fashion, the next pixel to the right won't work either because it doesn't have any neighbors above it. So, logically, the first pixel that you can do calculations on is at location (2,2) leaving a 1 pixel margin on all sides of the image and this resulting in <b>26 by 26</b> Output Size.